<a href="https://colab.research.google.com/github/basakstuff/DeepConvLSTM/blob/main/DCLSTM_TASK_A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task A: Multimodal activity recognition: Modes of locomotion

Including Relevent Library files.

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow import keras

from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
import time
import pickle as cp
from sliding_window import sliding_window

Loading Opportunity Dataset and including accessory files.

In [ ]:
#Loading Opportunity Dataset.
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00226/OpportunityUCIDataset.zip --no-check-certificate

In [ ]:
#Including preprocessing file.
!python preprocess_data.py -h

usage: preprocess_data.py [-h] -i INPUT -o OUTPUT [-t {gestures,locomotion}]

Preprocess OPPORTUNITY dataset

optional arguments:
  -h, --help            show this help message and exit
  -i INPUT, --input INPUT
                        OPPORTUNITY zip file
  -o OUTPUT, --output OUTPUT
                        Processed data file
  -t {gestures,locomotion}, --task {gestures,locomotion}
                        Type of activities to be recognized


## **TASK A: Applying Model on Locomotion Dataset (with NULL class)**

Preprocessing the data from data set.

In [ ]:
#Selecting the data relevent to TASK A (Detecting Locomotion).
!python preprocess_data.py -i data/OpportunityUCIDataset.zip -o oppChallenge_gestures.data -t locomotion

Checking dataset data/OpportunityUCIDataset.zip
Processing dataset files ...
... file OpportunityUCIDataset/dataset/S1-Drill.dat
... file OpportunityUCIDataset/dataset/S1-ADL1.dat
... file OpportunityUCIDataset/dataset/S1-ADL2.dat
... file OpportunityUCIDataset/dataset/S1-ADL3.dat
... file OpportunityUCIDataset/dataset/S1-ADL4.dat
... file OpportunityUCIDataset/dataset/S1-ADL5.dat
... file OpportunityUCIDataset/dataset/S2-Drill.dat
... file OpportunityUCIDataset/dataset/S2-ADL1.dat
... file OpportunityUCIDataset/dataset/S2-ADL2.dat
... file OpportunityUCIDataset/dataset/S2-ADL3.dat
... file OpportunityUCIDataset/dataset/S3-Drill.dat
... file OpportunityUCIDataset/dataset/S3-ADL1.dat
... file OpportunityUCIDataset/dataset/S3-ADL2.dat
... file OpportunityUCIDataset/dataset/S3-ADL3.dat
... file OpportunityUCIDataset/dataset/S2-ADL4.dat
... file OpportunityUCIDataset/dataset/S2-ADL5.dat
... file OpportunityUCIDataset/dataset/S3-ADL4.dat
... file OpportunityUCIDataset/dataset/S3-ADL5.dat
Fi

In [ ]:
# Number of Sensor Channels used in the OPPORTUNITY dataset.
NB_SENSOR_CHANNELS = 113

# Number of classes in which data is classified (or to be classified).
NUM_CLASSES = 5

# Length of the sliding window used to segmenting the time-series-data.
SLIDING_WINDOW_LENGTH = 24

# Steps of the sliding window used in segmenting the data.
SLIDING_WINDOW_STEP = 12

# Variable for Batch Size.
BATCH_SIZE = 100

# Number filters used in convolutional layers.
NUM_FILTERS = 64

# Size of filters used in convolutional layers.
FILTER_SIZE = 5

# Units in the long short-term recurrent layers.
NUM_UNITS_LSTM = 128

Loading the data and segmenting it according to length of sliding window.

In [ ]:
def load_dataset(filename):

    f = open(filename, 'rb')
    data = cp.load(f)
    f.close()

    X_train, y_train = data[0]
    X_test, y_test = data[1]

    print(" ..from file {}".format(filename))
    print(" ..reading instances: train {0}, test {1}".format(X_train.shape, X_test.shape))

    X_train = X_train.astype(np.float32)
    X_test = X_test.astype(np.float32)

    # The targets are casted to int8 for GPU compatibility.
    y_train = y_train.astype(np.uint8)
    y_test = y_test.astype(np.uint8)

    return X_train, y_train, X_test, y_test

print("Loading Data...")
X_train, y_train, X_test, y_test = load_dataset('data/oppChallenge_gestures.data')

assert NB_SENSOR_CHANNELS == X_train.shape[1]
def opp_sliding_window(data_x, data_y, ws, ss):
    data_x = sliding_window(data_x,(ws,data_x.shape[1]),(ss,1))
    data_y = np.asarray([[i[-1]] for i in sliding_window(data_y,ws,ss)])
    return data_x.astype(np.float32), data_y.reshape(len(data_y)).astype(np.uint8)

# Sensor data is segmented using a sliding window mechanism
X_test, y_test = opp_sliding_window(X_test, y_test, SLIDING_WINDOW_LENGTH, SLIDING_WINDOW_STEP)
print(" ..after sliding window (testing): inputs {0}, targets {1}".format(X_test.shape, y_test.shape))

# Data is reshaped since the input of the network is a 4 dimension tensor
X_test = X_test.reshape((-1, SLIDING_WINDOW_LENGTH, NB_SENSOR_CHANNELS,1))

Loading Data...
 ..from file data/oppChallenge_gestures.data
 ..reading instances: train (557963, 113), test (118750, 113)
 ..after sliding window (testing): inputs (9894, 24, 113), targets (9894,)


In [ ]:
X_train, y_train = opp_sliding_window(X_train, y_train, SLIDING_WINDOW_LENGTH, SLIDING_WINDOW_STEP)
print(" ..after sliding window (training): inputs {0}, targets {1}".format(X_train.shape, y_train.shape))
X_train = X_train.reshape((-1,SLIDING_WINDOW_LENGTH, NB_SENSOR_CHANNELS,1))
X_train.shape

 ..after sliding window (training): inputs (46495, 24, 113), targets (46495,)


(46495, 24, 113, 1)

In [ ]:
X_train.shape

(46495, 24, 113, 1)

Defining DeepConvLSTM Model. 

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
model = keras.Sequential()
model.add(keras.Input(shape=(SLIDING_WINDOW_LENGTH, NB_SENSOR_CHANNELS,1)))

#intializing randomly orthogonal weights.
initializer = tf.keras.initializers.Orthogonal()

#Adding 4 layes of CNN.
model.add(layers.Conv2D(NUM_FILTERS, kernel_size=(FILTER_SIZE, 1), activation="relu",kernel_initializer=initializer))
model.add(layers.Conv2D(NUM_FILTERS, kernel_size=(FILTER_SIZE, 1), activation="relu",kernel_initializer=initializer))
model.add(layers.Conv2D(NUM_FILTERS, kernel_size=(FILTER_SIZE, 1), activation="relu",kernel_initializer=initializer))
model.add(layers.Conv2D(NUM_FILTERS, kernel_size=(FILTER_SIZE, 1), activation="relu",kernel_initializer=initializer))
model.add(layers.Permute((2,1,3)))
model.add(layers.Reshape( (int(model.layers[4].output_shape[1]), int(model.layers[4].output_shape[2]) * int(model.layers[4].output_shape[3]))))

#Adding 2 layers of LSTM.
model.add(layers.LSTM(NUM_UNITS_LSTM,dropout=0.5,return_sequences=True,kernel_initializer=initializer))
model.add(layers.LSTM(NUM_UNITS_LSTM,dropout=0.5,return_sequences=True,kernel_initializer=initializer))
model.add(layers.Flatten())

#Applying dense layer of Softmax to the output of 4-CNN layers and 2 LSTM layers.
model.add(layers.Dense(NUM_CLASSES, activation="softmax"))
    
#Printing Summery of Model.
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 20, 113, 64)       384       
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 16, 113, 64)       20544     
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 12, 113, 64)       20544     
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 8, 113, 64)        20544     
_________________________________________________________________
permute_1 (Permute)          (None, 113, 8, 64)        0         
_________________________________________________________________
reshape_1 (Reshape)          (None, 113, 512)          0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 113, 128)         

In [ ]:
model.layers[4].output_shape


(None, 113, 8, 64)

Encoding the training and testing data to One-Hot Encoded form.

In [ ]:
from sklearn.preprocessing import OneHotEncoder
def prepare_targets(y_train, y_test):
	ohe = OneHotEncoder()
	ohe.fit(y_train)
	y_train_enc = ohe.transform(y_train)
	y_test_enc = ohe.transform(y_test)
	return y_train_enc.A, y_test_enc.A

In [ ]:
y_train=y_train.reshape(-1,1)
y_test=y_test.reshape(-1,1)
y_train_enc, y_test_enc = prepare_targets(y_train, y_test)

Compiling Model

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='RMSprop', metrics=[tf.keras.metrics.CategoricalAccuracy()])

In [ ]:
y_train.shape

(46495, 1)

Training Model

In [ ]:
#model.fit(X_train, y_train_enc, batch_size=BATCH_SIZE, epochs=100,validation_split=0.1)
#model.save("MODEL_NAME.h5")

#The model is already trained using above statement and trained model is loaded for testing data.

Epoch 1/100
210/210 [==============================] - 33s 123ms/step - loss: 1.1810 - categorical_accuracy: 0.4906 - val_loss: 0.9097 - val_categorical_accuracy: 0.6656
Epoch 2/100
210/210 [==============================] - 25s 120ms/step - loss: 0.5749 - categorical_accuracy: 0.7835 - val_loss: 0.8247 - val_categorical_accuracy: 0.6877
Epoch 3/100
210/210 [==============================] - 26s 123ms/step - loss: 0.4671 - categorical_accuracy: 0.8253 - val_loss: 0.6559 - val_categorical_accuracy: 0.7781
Epoch 4/100
210/210 [==============================] - 26s 124ms/step - loss: 0.3562 - categorical_accuracy: 0.8678 - val_loss: 0.5546 - val_categorical_accuracy: 0.8062
Epoch 5/100
210/210 [==============================] - 26s 123ms/step - loss: 0.3218 - categorical_accuracy: 0.8767 - val_loss: 0.6324 - val_categorical_accuracy: 0.8155
Epoch 6/100
210/210 [==============================] - 26s 123ms/step - loss: 0.2843 - categorical_accuracy: 0.8954 - val_loss: 0.6185 - val_categoric

Load Model.

In [ ]:
#You can load your model.

from tensorflow.keras.models import load_model

new_model1 = load_model('Null_Basak.h5') #(MODEL_NAME.h5)


Testing our Model on test data.

In [ ]:
y_prob = new_model1.predict(X_test) 
y_classes = y_prob.argmax(axis=-1)

Printing F1 Score and relevent Result Metrics.

In [ ]:
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import confusion_matrix
from sklearn.metrics import matthews_corrcoef

precision, recall, f1_score, supp = precision_recall_fscore_support(y_classes, y_test, average='micro')


print(confusion_matrix(y_classes, y_test))
print("PRECISION: {0}" .format(precision))
print("RECALL: {0}".format(recall))
print("F1_SCORE : {0}".format(f1_score))
print("Matthews:")
print(matthews_corrcoef(y_classes, y_test)

[[1487   29  123   17   15]
 [ 208 2842  371   28    3]
 [ 278  209 1776    3    0]
 [  61   21    2 1963   18]
 [   8    0    0    5  427]]
PRECISION: 0.8586011724277339
RECALL: 0.8586011724277339
F1_SCORE : 0.8586011724277339
0.8150721926225444


In [ ]:
y_test

array([[0],
       [0],
       [0],
       ...,
       [0],
       [0],
       [0]], dtype=uint8)

## **Task A: Applying Model on Locomotion Dataset (Without NULL class)**

Preprocessing the data from data set.

In [ ]:
!python preprocess_data.py -i data/OpportunityUCIDataset.zip -o oppChallenge_gestures.data -t locomotion

Checking dataset data/OpportunityUCIDataset.zip
Processing dataset files ...
... file OpportunityUCIDataset/dataset/S1-Drill.dat
... file OpportunityUCIDataset/dataset/S1-ADL1.dat
... file OpportunityUCIDataset/dataset/S1-ADL2.dat
... file OpportunityUCIDataset/dataset/S1-ADL3.dat
... file OpportunityUCIDataset/dataset/S1-ADL4.dat
... file OpportunityUCIDataset/dataset/S1-ADL5.dat
... file OpportunityUCIDataset/dataset/S2-Drill.dat
... file OpportunityUCIDataset/dataset/S2-ADL1.dat
... file OpportunityUCIDataset/dataset/S2-ADL2.dat
... file OpportunityUCIDataset/dataset/S2-ADL3.dat
... file OpportunityUCIDataset/dataset/S3-Drill.dat
... file OpportunityUCIDataset/dataset/S3-ADL1.dat
... file OpportunityUCIDataset/dataset/S3-ADL2.dat
... file OpportunityUCIDataset/dataset/S3-ADL3.dat
... file OpportunityUCIDataset/dataset/S2-ADL4.dat
... file OpportunityUCIDataset/dataset/S2-ADL5.dat
... file OpportunityUCIDataset/dataset/S3-ADL4.dat
... file OpportunityUCIDataset/dataset/S3-ADL5.dat
Fi

Defining important variables.

In [ ]:
# Variable for Number of sensor channels in the OPPORTUNITY dataset.
NB_SENSOR_CHANNELS = 113

# Variable for number of classes in which data is classified (or to be classified). 
NUM_CLASSES = 4

# Variable for length of the sliding window used in segmenting the time-series-data.
SLIDING_WINDOW_LENGTH = 24

# Variable for steps of the sliding window used in segmenting the data
SLIDING_WINDOW_STEP = 12

# Variable for Batch Size
BATCH_SIZE = 200

# Number filters used in convolutional layers
NUM_FILTERS = 92

# Size of each filter convolutional layers
FILTER_SIZE = 5

# Number of unit in the LSTM layer.
NUM_UNITS_LSTM = 128

Loading Data an segmenting data according to sliding window length.

In [ ]:
def load_dataset(filename):

    f = open(filename, 'rb')
    data = cp.load(f)
    f.close()

    X_train, y_train = data[0]
    X_test, y_test = data[1]

    print(" ..from file {}".format(filename))
    print(" ..reading instances: train {0}, test {1}".format(X_train.shape, X_test.shape))

    X_train = X_train.astype(np.float32)
    X_test = X_test.astype(np.float32)

    # The targets are casted to int8 for GPU compatibility.
    y_train = y_train.astype(np.uint8)
    y_test = y_test.astype(np.uint8)

    return X_train, y_train, X_test, y_test

print("Loading data...")
X_train, y_train, X_test, y_test = load_dataset('data/oppChallenge_gestures.data')

assert NB_SENSOR_CHANNELS == X_train.shape[1]
def opp_sliding_window(data_x, data_y, ws, ss):
    data_x = sliding_window(data_x,(ws,data_x.shape[1]),(ss,1))
    data_y = np.asarray([[i[-1]] for i in sliding_window(data_y,ws,ss)])
    return data_x.astype(np.float32), data_y.reshape(len(data_y)).astype(np.uint8)

# Sensor data is segmented using a sliding window mechanism
X_test, y_test = opp_sliding_window(X_test, y_test, SLIDING_WINDOW_LENGTH, SLIDING_WINDOW_STEP)
print(" ..after sliding window (testing): inputs {0}, targets {1}".format(X_test.shape, y_test.shape))

# Data is reshaped since the input of the network is a 4 dimension tensor
X_test = X_test.reshape((-1, SLIDING_WINDOW_LENGTH, NB_SENSOR_CHANNELS,1))

Loading data...
 ..from file data/oppChallenge_gestures.data
 ..reading instances: train (557963, 113), test (118750, 113)
 ..after sliding window (testing): inputs (9894, 24, 113), targets (9894,)


In [ ]:
X_train, y_train = opp_sliding_window(X_train, y_train, SLIDING_WINDOW_LENGTH, SLIDING_WINDOW_STEP)
print(" ..after sliding window (training): inputs {0}, targets {1}".format(X_train.shape, y_train.shape))
X_train = X_train.reshape((-1,SLIDING_WINDOW_LENGTH, NB_SENSOR_CHANNELS,1))
print(X_train.shape)
print(y_train.shape)

 ..after sliding window (training): inputs (46495, 24, 113), targets (46495,)
(46495, 24, 113, 1)
(46495,)


Defining Model.

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
model = keras.Sequential()
model.add(keras.Input(shape=(SLIDING_WINDOW_LENGTH, NB_SENSOR_CHANNELS,1)))

# intializing randomly orthogonal weights.
initializer = tf.keras.initializers.Orthogonal()

# Adding 4 layers of CNN
model.add(layers.Conv2D(NUM_FILTERS, kernel_size=(FILTER_SIZE, 1), activation="relu", kernel_initializer = initializer))
model.add(layers.Conv2D(NUM_FILTERS, kernel_size=(FILTER_SIZE, 1), activation="relu",kernel_initializer = initializer))
model.add(layers.Conv2D(NUM_FILTERS, kernel_size=(FILTER_SIZE, 1), activation="relu",kernel_initializer = initializer))
model.add(layers.Conv2D(NUM_FILTERS, kernel_size=(FILTER_SIZE, 1), activation="relu",kernel_initializer = initializer))
model.add(layers.Permute((2,1,3)))
model.add(layers.Reshape( (int(model.layers[4].output_shape[1]), int(model.layers[4].output_shape[2]) * int(model.layers[4].output_shape[3]))))

# Adding 2 layers of LSTM.
model.add(layers.LSTM(NUM_UNITS_LSTM, dropout=0.5,return_sequences=True,kernel_initializer = initializer))
model.add(layers.LSTM(NUM_UNITS_LSTM, dropout=0.5,return_sequences=True,kernel_initializer = initializer))
model.add(layers.Flatten())

# Adding a dense layer of softmax.
model.add(layers.Dense(NUM_CLASSES, activation="softmax"))

# Printing Model Summary.
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 20, 113, 92)       552       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 113, 92)       42412     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 12, 113, 92)       42412     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 8, 113, 92)        42412     
_________________________________________________________________
permute (Permute)            (None, 113, 8, 92)        0         
_________________________________________________________________
reshape (Reshape)            (None, 113, 736)          0         
_________________________________________________________________
lstm (LSTM)                  (None, 113, 128)          4

Encoding the training and testing data to One-Hot Encoded form.

In [ ]:
from sklearn.preprocessing import OneHotEncoder
def prepare_targets(y_train, y_test):
	ohe = OneHotEncoder()
	ohe.fit(y_train)
	y_train_enc = ohe.transform(y_train)
	y_test_enc = ohe.transform(y_test)
	return y_train_enc.A, y_test_enc.A

Removing NULL class values.

In [ ]:
y_train=y_train.reshape(-1,1)
y_test=y_test.reshape(-1,1)

count = 0
idx = []
for i in range(0,y_train.shape[0]):
  if(y_train[i] == 0):
    count += 1
    idx.append(i)
print(len(idx))
y_train_new = np.delete(y_train,idx)
y_train_new = y_train_new.reshape(-1,1)
print(y_train_new.shape)

print(X_train.shape)
X_train_new = np.delete(X_train,idx,axis=0)
print(X_train_new.shape)

count = 0
idx = []
for i in range(0,y_test.shape[0]):
  if(y_test[i] == 0):
    count += 1
    idx.append(i)
print(len(idx))
y_test_new = np.delete(y_test,idx,axis = 0)
print(y_test_new.shape)

X_test_new = np.delete(X_test,idx,axis = 0)
print(X_test_new.shape)

y_train_enc, y_test_enc = prepare_targets(y_train_new, y_test_new)
print(y_train_enc.shape)
print(y_test_enc.shape)

7680
(38815, 1)
(46495, 24, 113, 1)
(38815, 24, 113, 1)
2042
(7852, 1)
(7852, 24, 113, 1)
(38815, 4)
(7852, 4)


In [ ]:
y_train_enc

array([[0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       ...,
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.]])

Compiling Model

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='RMSprop', metrics=[tf.keras.metrics.CategoricalAccuracy()])

In [ ]:
X_train.shape

(46495, 24, 113, 1)

Training Model

In [ ]:
model.fit(X_train_new, y_train_enc, batch_size=BATCH_SIZE, epochs=100,validation_split=0.1)
model.save("MODEL_NAME.h5")

#Model is already trained using above lines of code and loaded directly for testing purposes.

Epoch 1/100
175/175 [==============================] - 72s 191ms/step - loss: 1.3217 - categorical_accuracy: 0.5308 - val_loss: 0.9058 - val_categorical_accuracy: 0.6932
Epoch 2/100
175/175 [==============================] - 31s 177ms/step - loss: 0.4552 - categorical_accuracy: 0.8209 - val_loss: 0.4024 - val_categorical_accuracy: 0.8583
Epoch 3/100
175/175 [==============================] - 32s 182ms/step - loss: 0.3506 - categorical_accuracy: 0.8654 - val_loss: 0.3676 - val_categorical_accuracy: 0.8756
Epoch 4/100
175/175 [==============================] - 31s 179ms/step - loss: 0.3032 - categorical_accuracy: 0.8825 - val_loss: 0.3033 - val_categorical_accuracy: 0.8903
Epoch 5/100
175/175 [==============================] - 31s 180ms/step - loss: 0.2987 - categorical_accuracy: 0.8891 - val_loss: 0.3956 - val_categorical_accuracy: 0.8594
Epoch 6/100
175/175 [==============================] - 32s 181ms/step - loss: 0.2570 - categorical_accuracy: 0.8974 - val_loss: 0.2817 - val_categoric

Loading Model.

In [ ]:
from tensorflow.keras.models import load_model

new_model2 = load_model('NoNull_Batch200_Epoch100_Filter92.h5') #(MODEL_NAME.h5)

Testing our Model on test data.

In [ ]:
y_prob = new_model2.predict(X_test_new) 
#print(y_prob)

#adjusting format of output to the required format of result.
y_classes = y_prob.argmax(axis=-1)

for i in range(0,y_classes.shape[0]):
  y_classes[i] += 1
print(np.sum(y_classes))
print(np.sum(y_test_new))

15368
15545


In [ ]:
y_prob.shape

(7852, 4)

Printing F1 Score and relevent Result Metrics.

In [ ]:
from sklearn.metrics import precision_recall_fscore_support

precision, recall, f1_score, supp = precision_recall_fscore_support(y_test_new,y_classes,average='weighted')

from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_classes, y_test_new))
#print(metrics)
print("PRECISION: {0}" .format(precision))
print("RECALL: {0}".format(recall))
print("F1_SCORE : {0}".format(f1_score))
print("Matthews:")
print(matthews_corrcoef(y_classes, y_test)

[[2859  359   42    1]
 [ 211 1911    3    2]
 [  31    2 1961    9]
 [   0    0   10  451]]
PRECISION: 0.9152678389250153
RECALL: 0.9146714212939379
F1_SCORE : 0.9144686507125466


**----------------------------------------------------------------------------------------------------------------------------------------------------------------**